# Shohei Ohtani Batting Stats

- 大谷翔平の打撃成績を可視化する例
- サンプルデータはkm/h & m 単位になっていますが, [オリジナルデータ](https://baseballsavant.mlb.com/statcast_search)はmile/h & feetなのでご注意ください

In [8]:
import pandas as pd

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [9]:
# データ読み込み. game_dateは予めデータ型を変更

df = pd.read_csv('dataset/20220919_baseball-savant-shohei-ohtani-atbat.csv', header=0)

df['game_date'] = pd.to_datetime(df['game_date'])
df.head(10)

,game_date,game_month,game_pk,inning,pitcher_name,p_throws,batter_name,stand,at_bat_number,pitch_number,pitch_name,effective_speed,release_spin_rate,release_extension,type,events,description,des,zone,bb_type,launch_speed_angle,hit_location,launch_speed,launch_angle,hit_distance_sc,pfx_x,pfx_z,plate_x,plate_z,release_pos_x,release_pos_z,effective_speed_kh,release_extension_cm,pfx_x_cm,pfx_z_cm,plate_x_cm,plate_z_cm,release_pos_x_cm,release_pos_z_cm,launch_speed_angle_category,launch_speed_kh,hit_distance_sc_m,hc_x,hc_y,hc_x_0,hc_y_0,hc_x_ft,hc_y_ft,hc_x_0_ft,hc_y_0_ft,hc_x_m,hc_y_m,hc_x_0_m,hc_y_0_m,events_summary
0,2022-09-17,9,663426,1,"Kirby, George",R,"Ohtani, Shohei",L,6,1,4-Seam Fastball,95.8,2280.0,6.3,X,double,hit_into_play,Shohei Ohtani doubles (26) on a sharp fly ball...,4.0,fly_ball,6.0,NaN,106.8,26.0,371.0,-1.02,1.22,-0.40,2.75,-1.29,5.98,153.28,192.02,-31.09,37.19,-12.19,83.82,-39.32,182.27,Barrel,170.88,113.08,36.46,77.59,-88.96,120.68,91.15,193.98,-222.40,301.70,27.78,59.12,-67.79,91.96,hits
1,2022-09-17,9,663426,4,"Kirby, George",R,"Ohtani, Shohei",L,26,1,4-Seam Fastball,95.1,2140.0,6.4,S,NaN,called_strike,Shohei Ohtani walks.,12.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.75,1.24,0.38,3.59,-1.28,6.01,152.16,195.07,-22.86,37.80,11.58,109.42,-39.01,183.18,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
2,2022-09-17,9,663426,4,"Kirby, George",R,"Ohtani, Shohei",L,26,2,Sinker,95.1,1910.0,6.5,B,NaN,ball,Shohei Ohtani walks.,13.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.42,0.82,-1.43,2.33,-1.09,5.93,152.16,198.12,-43.28,24.99,-43.59,71.02,-33.22,180.75,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
3,2022-09-17,9,663426,4,"Kirby, George",R,"Ohtani, Shohei",L,26,3,4-Seam Fastball,96.3,2189.0,6.4,B,NaN,ball,Shohei Ohtani walks.,12.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.71,1.29,0.19,3.80,-1.14,6.02,154.08,195.07,-21.64,39.32,5.79,115.82,-34.75,183.49,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
4,2022-09-17,9,663426,4,"Kirby, George",R,"Ohtani, Shohei",L,26,4,Changeup,88.2,1509.0,6.8,B,NaN,ball,Shohei Ohtani walks.,9.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.35,0.32,0.53,1.55,-1.30,5.61,141.12,207.26,-41.15,9.75,16.15,47.24,-39.62,170.99,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
5,2022-09-17,9,663426,4,"Kirby, George",R,"Ohtani, Shohei",L,26,5,Sinker,95.2,2300.0,6.7,B,walk,ball,Shohei Ohtani walks.,14.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.32,0.84,1.11,1.98,-1.10,5.78,152.32,204.22,-40.23,25.60,33.83,60.35,-33.53,176.17,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk
6,2022-09-17,9,663426,6,"Kirby, George",R,"Ohtani, Shohei",L,42,1,Changeup,89.7,1540.0,6.8,B,NaN,ball,"Shohei Ohtani grounds out, shortstop J. P. Cr...",14.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.23,0.54,0.64,1.26,-1.26,5.70,143.52,207.26,-37.49,16.46,19.51,38.40,-38.40,173.74,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
7,2022-09-17,9,663426,6,"Kirby, George",R,"Ohtani, Shohei",L,42,2,Changeup,88.7,1675.0,6.7,X,field_out,hit_into_play,"Shohei Ohtani grounds out, shortstop J. P. Cr...",7.0,ground_ball,4.0,NaN,95.0,4.0,72.0,-1.23,0.48,-0.65,1.59,-1.35,5.67,141.92,204.22,-37.49,14.63,-19.81,48.46,-41.15,172.82,Flare/Burner,152.00,21.95,138.17,135.77,12.75,62.50,345.42,339.43,31.87,156.25,105.29,103.46,9.72,47.63,other
8,2022-09-16,9,663427,1,"Ray, Robbie",L,"Ohtani, Shohei",L,7,1,4-Seam Fastball,94.2,2186.0,6.6,X,double,hit_into_play,Shohei Ohtani doubles (25) on a fly ball to le...,NaN,fly_ball,3.0,NaN,92.0,61.0,205.0,0.78,1.42,0.27,2.97,2.59,6.17,150.72,201.17,23.77,43.28,8.23,90.53,78.94,188.06,Under,147.20,62.48,68.24,135.43,-57.18,62.84,170.60,338.58,-142.95,157.10,52.00,103.20,-43.57,47.88,hits
9,2022-09-16,9,663427,3,"Ray, Robbie",L,"Ohtani, Shohei",L,23,1,4-Seam Fastball,92.4,2334.0,6.6,X,field_out,hit_into_play,Shohei Ohtani flies out to left fielder Jesse ...,NaN,fly_ball,3.0,NaN,94.2,59.0,219.0,0.59,1.58,-0.36,2.61,2.54,6.19,147.84,201.17,17.98,48.16,-10.97,79.55,77.42,188.67,Un

In [10]:
# 念の為データ型を確認
df.dtypes

game_date                      datetime64[ns]
game_month                              int64
game_pk                                 int64
inning                                  int64
pitcher_name                           object
p_throws                               object
batter_name                            object
stand                                  object
at_bat_number                           int64
pitch_number                            int64
pitch_name                             object
effective_speed                       float64
release_spin_rate                     float64
release_extension                     float64
type                                   object
events                                 object
description                            object
des                                    object
zone                                  float64
bb_type                                object
launch_speed_angle                    float64
hit_location                      

## 投球データの可視化

- どこに投げられているか
- 球種毎, 結果毎にプロット

In [11]:
# グラフ描画用の関数

import plotly.express as px
import plotly.graph_objects as go

# ゾーンの横幅と縦幅
PLATE_ZONE_X = (-100, 100)
PLATE_ZONE_Y = (0, 225)

# ストライクゾーン定義（本当は上下可変ですがわかりやすいように固定にしています）
PLATE_STRIKE_ZONE_X = [-28.9, -28.9, 28.9, 28.9, -28.9]
PLATE_STRIKE_ZONE_Y = [48, 104, 104, 48, 48]

HOVER = [
    "game_date",
    "batter_name",
    "pitcher_name",
    "effective_speed_kh",
    "pitch_name",
    "hit_distance_sc_m",
    "launch_speed_kh",
    "launch_angle",
    "des",
    "description"
]
LEGEND = dict(
    yanchor="top",
    y=1,
    xanchor="left",
    x=0,
    orientation="h"
)
def strike_zone(name: str = 'strike zone') -> go.Scatter:
    return go.Scatter(
        name=name,
        x=PLATE_STRIKE_ZONE_X,
        y=PLATE_STRIKE_ZONE_Y,
        line_color='red'
    )


def plate(df: pd.DataFrame, title: str = 'Pitch on plate(Umpire view)',
            hover: str = 'pitch_name') -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="plate_x_cm",
        y="plate_z_cm",
        color=hover,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.add_trace(strike_zone())
    fig.update_layout(
        title=title,
        autosize=False,
        width=800,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=PLATE_ZONE_X, title='plate_x(cm)', dtick=25)
    fig.update_yaxes(range=PLATE_ZONE_Y, title='plate_x(cm)', dtick=25)
    return fig


In [12]:
# 早速可視化してみる

_fig = plate(df)
_fig.show()

In [13]:
# イベントごとの結果

_fig = plate(df, hover='events')
_fig.show()

In [14]:
# 一試合に絞る

_fig = plate(df[df['game_date'] == '2022-09-17'])
_fig.show()